In [37]:
import csv
import time
import requests
from bs4 import BeautifulSoup
from pattern.en import ngrams
from textblob import TextBlob
from textblob import classifiers
import pandas as pd

In [58]:
PATH1 = 'data/SentimentData/amazon_cells_labelled.txt'
PATH2 = 'data/SentimentData/imdb_labelled.txt'
PATH3 = 'data/SentimentData/yelp_labelled.txt'

amazon_data = pd.read_csv(PATH1, sep='\t', header=None, names=['sentence', 'sentiment'])
imdb_data = pd.read_csv(PATH2, sep='\t', header=None, names=['sentence', 'sentiment'])
yelp_data = pd.read_csv(PATH3, sep='\t', header=None, names=['sentence', 'sentiment'])

In [106]:
total_data = amazon_data
total_data = total_data.append(imdb_data, ignore_index=True)
total_data = total_data.append(yelp_data, ignore_index=True)

In [144]:
train_data_full = [tuple(x) for x in total_data.values]
train_data = train_data_full[:2000]
print(len(train_data))

2748
2000


In [145]:
print('training NLP classifier...')
classifier = classifiers.NaiveBayesClassifier(train_data)
classifier.train()
print('done')

training NLP classifier...
done


In [146]:
print('classifying...')
prob_dist = classifier.prob_classify("Stocks are terrible")
print('result')
print(prob_dist.prob(0))

classifying...
result
0.6754882030498195


In [147]:
classifier.accuracy(train_data_full[2000:])

0.7593582887700535

In [148]:
from googlesearch import search
# to search 

companies = {'Synchrony Financial': [],
            '3M': [],
            'Bayer': [],
            'Honeywell': []}


for company in companies:
    query = company + " News"
    for link in search(query, num=5, stop=1, pause=2): 
        companies[company].append(link)

In [149]:
# Open the choosen news articles and extract the title
title_to_polarity = {}
net_company_scores = {}
for company in companies:
    net_company_scores[company] = 0
    count = 0
    for link in companies[company]:
        count += 1
        results = requests.get(link)
        results_text = BeautifulSoup(results.text, 'html.parser')
        try:
            title = results_text.title.string
            prob_1 = classifier.prob_classify(title).prob(1)
            title_to_polarity[title] = prob_1
            net_company_scores[company] += prob_1
        except:
            continue
    if count:
        net_company_scores[company] /= count

print(net_company_scores)

7
7
7
7
{'Synchrony Financial': 4.091920365875385, '3M': 3.2328777948018086, 'Bayer': 4.11913610995684, 'Honeywell': 4.265869258906456}


In [34]:
text = '''
The titular threat of The Blob has always struck me as the ultimate movie
monster: an insatiably hungry, amoeba-like mass able to penetrate
virtually any safeguard, capable of--as a doomed doctor chillingly
describes it--"assimilating flesh on contact.
Snide comparisons to gelatin be damned, it's a concept with the most
devastating of potential consequences, not unlike the grey goo scenario
proposed by technological theorists fearful of
artificial intelligence run rampant.
'''

blob = TextBlob(text)
blob.tags           # [('The', 'DT'), ('titular', 'JJ'),
                    #  ('threat', 'NN'), ('of', 'IN'), ...]

blob.noun_phrases   # WordList(['titular threat', 'blob',
                    #            'ultimate movie monster',
                    #            'amoeba-like mass', ...])

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

0.06000000000000001
-0.34166666666666673
